In [1]:
import pandas as pd

In [2]:
gdf = pd.read_csv('refGene_hg19.txt', sep='\t', header=None)

gdf['chrom'] = gdf[2]
gdf['start'] = gdf[4]
gdf['end']   = gdf[5]
gdf['gene']  = gdf[12]
gdf = gdf[['chrom', 'start', 'end', 'gene']]

In [3]:
pheno = pd.read_csv('../16p12.2_rnaseq_analysis/data/pheno_final.tsv', sep='\t')

samples = list(pheno.subject.unique())

In [4]:
with open('survivor/all_codes.list', 'r') as f:
    allsamples = f.readlines()
allsamples = [s.strip() for s in allsamples]

In [5]:
df_small = pd.read_csv('output/merged/allsamples.annotated.tsv', sep='\t')
df_small = df_small[df_small.intra_cohort_count < 10]
df_small = df_small.drop(['intra_cohort_count', 'intra_cohort_freq'], axis=1)

In [6]:
df_large = pd.read_csv('output/merged_greater_50/merged.tsv', sep='\t')



In [7]:
cnvs = df_small.append(df_large)
cnvs = cnvs[cnvs.subject.isin(pheno.subject)].copy()

# sort cnvs by gene

In [8]:
cnvs = cnvs[~(cnvs.genes.isna())]

In [17]:
genec = pd.DataFrame()
genec = []
cols = ['chrom', 'start', 'end', 'svtype', 'caller', 'num_callers', 'length', 'subject']

for i, row in cnvs.iterrows():
    genes = row['genes']
    genes = genes.split(';')
    
    for gene in genes:
        genec.append([gene] + list(row[cols]))


In [18]:
genec = pd.DataFrame(genec, columns=['gene', 'chrom', 'start', 'end', 'svtype', 'caller', 'num_callers', 'length', 'subject'])

In [11]:
genec.to_csv('output/merged/16p_all_rare_cnvs.tsv', sep='\t', index=False)

# mark encapsulated

In [26]:
gdf = pd.read_csv('refGene_hg19.txt', sep='\t', header=None)

gdf['chrom'] = gdf[2]
gdf['start'] = gdf[4]
gdf['end']   = gdf[5]
gdf['gene']  = gdf[12]
gdf['strand']  = gdf[3]
gdf = gdf[['chrom', 'start', 'end', 'gene', 'strand']]
gdf = gdf.set_index('gene', drop=False)

In [42]:

for i, row in genec.iterrows():
    gene = row['gene']
    start = row['start']
    end = row['end']
    
    rowg = gdf.loc[gene]
    startg = rowg['start']
    endg = rowg['end']
    strand = rowg['strand']
    
    genec.at[i, 'gene_start'] = startg
    genec.at[i, 'gene_end'] = endg
    genec.at[i, 'strand'] = strand
    
    if start < startg and endg < end:
        genec.at[i, 'gene_covered'] = 'encapsulated'
    if startg < start and end < endg:
        genec.at[i, 'gene_covered'] = 'interstitial'
    if strand == '+':
        if start < startg and end < endg:
            genec.at[i, 'gene_covered'] = '5prime'
        if startg < start and endg < end:
            genec.at[i, 'gene_covered'] = '3prime'
    if strand == '-':
        if startg < start and endg < end:
            genec.at[i, 'gene_covered'] = '5prime'
        if start < startg and end < endg:
            genec.at[i, 'gene_covered'] = '3prime'

In [46]:
genec.to_csv('output/merged/16p_all_rare_cnvs.tsv', sep='\t', index=False)